## Preparo dos dados

#### Através deste job, os dados presentes no arquivo csv de filmes se tornam limpos e confiáveis para as etapas posteriores. A limpeza de dados realizada neste job seleciona apenas as colunas e linhas que serão necessárias para responder às perguntas que foram previamente definidas. Ou seja, nem todos todos os dados são mantidos, apenas aqueles que são relevantes para atender aos objetivos analíticos.

#### O arquivo csv já havia sido previamente catalogado previamente no AWS Glue por meio de um crawler.

In [1]:
%idle_timeout 60
%glue_version 3.0 # versão do AWS Glue que o job utilizará
%worker_type G.1X #  tipo de worker a ser usado
%number_of_workers 2 # número de workers que serão alocados para o job

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

#  configurando o ambiente do Glue, criando instâncias do SparkContext, GlueContext e um objeto de job
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.38.1 
Current idle_timeout is 2800 minutes.
idle_timeout has been set to 60 minutes.
Setting Glue version to: 3.0
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 5
Setting new number of workers to: 2
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::965478153338:role/service-role/AWSGlueServiceRole
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 2
Session ID: f516d46e-cae5-4776-9610-8230a17a2143
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.38.1
--enable-glue-datacatalog true
Waiting for session f

In [15]:
from pyspark.sql.functions import col, when, substring, lower, sum, count
from awsglue.dynamicframe import DynamicFrame

In [3]:
# criando um DynamicFrame a partir da tabela dos dados no AWS Glue Data Catalog e exibindo seu esquema
dyf = glueContext.create_dynamic_frame.from_catalog(database='movies-raw-data', table_name='local', separator="|")
dyf.printSchema()

root
|-- id: string
|-- titulopincipal: string
|-- titulooriginal: string
|-- anolancamento: choice
|    |-- long
|    |-- string
|-- tempominutos: string
|-- genero: string
|-- notamedia: choice
|    |-- double
|    |-- string
|-- numerovotos: choice
|    |-- long
|    |-- string
|-- generoartista: string
|-- personagem: string
|-- nomeartista: string
|-- anonascimento: string
|-- anofalecimento: string
|-- profissao: string
|-- titulosmaisconhecidos: string
|-- partition_0: string
|-- partition_1: string
|-- partition_2: string
|-- anotermino: string


In [4]:
# convertendo o dynamicframe para um dataframe spark
df_movies = dyf.toDF()
df_movies.show(5)

+---------+--------------+--------------+-------------+------------+-------+-----------+-----------+-------------+--------------------+-----------------+-------------+--------------+------------+---------------------+-----------+-----------+-----------+----------+
|       id|titulopincipal|titulooriginal|anolancamento|tempominutos| genero|  notamedia|numerovotos|generoartista|          personagem|      nomeartista|anonascimento|anofalecimento|   profissao|titulosmaisconhecidos|partition_0|partition_1|partition_2|anotermino|
+---------+--------------+--------------+-------------+------------+-------+-----------+-----------+-------------+--------------------+-----------------+-------------+--------------+------------+---------------------+-----------+-----------+-----------+----------+
|tt0000009|    Miss Jerry|    Miss Jerry| {1894, null}|          45|Romance|{5.3, null}|{200, null}|      actress|Miss Geraldine Ho...|  Blanche Bayliss|         1878|          1951|     actress|          

#### As colunas do dataframe que não serão necessárias às análises são excluídas

In [5]:
# lista de nomes das colunas para remover
colunas_remover_movies = ['tituloOriginal', 'personagem', 'generoArtista', 'nomeArtista', 'profissao', 'anoNascimento', 'anoFalecimento', 'titulosMaisConhecidos','partition_0','partition_1','partition_2','anotermino']

# método drop para remover as colunas
df_movies = df_movies.drop(*colunas_remover_movies)

#### Na coluna contendo os ids dos filmes os dados estão iniciando com "tt". No código a seguir esses dois caracteres iniciais são desconsiderados e os campos passam a conter apenas os números.

In [6]:
df_movies = df_movies.withColumn('id', substring(col('id'), 3, 100))

In [7]:
df_movies.show(5)

+-------+--------------+-------------+------------+-------+-----------+-----------+
|     id|titulopincipal|anolancamento|tempominutos| genero|  notamedia|numerovotos|
+-------+--------------+-------------+------------+-------+-----------+-----------+
|0000009|    Miss Jerry| {1894, null}|          45|Romance|{5.3, null}|{200, null}|
|0000009|    Miss Jerry| {1894, null}|          45|Romance|{5.3, null}|{200, null}|
|0000009|    Miss Jerry| {1894, null}|          45|Romance|{5.3, null}|{200, null}|
|0000502|      Bohemios| {1905, null}|         100|     \N|{4.2, null}| {14, null}|
|0000502|      Bohemios| {1905, null}|         100|     \N|{4.2, null}| {14, null}|
+-------+--------------+-------------+------------+-------+-----------+-----------+
only showing top 5 rows


In [8]:
# Obtendo o número de linhas (registros)
num_linhas = df_movies.count()
# Obtendo o número de colunas
num_colunas = len(df_movies.columns)

# Exiba o número de linhas e colunas
print("Número de Linhas:", num_linhas)
print("Número de Colunas:", num_colunas)

Número de Linhas: 1513944
Número de Colunas: 7


#### Como as análises serão restritas a filmes do gênero "crime", nessa parte do código são selecionados apenas os filmes que contêm este gênero. Para garantir que não haja problemas na filtragem dos dados, todos os caracteres são passados para letras minúsculas.

In [9]:
# Usando a função lower para colocar todos os caracteres da coluna "genero" em letras minúsculas
df_movies = df_movies.withColumn('genero', lower(df_movies['genero']))

df_movies.show()

+-------+--------------------+-------------+------------+--------------------+-----------+-----------+
|     id|      titulopincipal|anolancamento|tempominutos|              genero|  notamedia|numerovotos|
+-------+--------------------+-------------+------------+--------------------+-----------+-----------+
|0000009|          Miss Jerry| {1894, null}|          45|             romance|{5.3, null}|{200, null}|
|0000009|          Miss Jerry| {1894, null}|          45|             romance|{5.3, null}|{200, null}|
|0000009|          Miss Jerry| {1894, null}|          45|             romance|{5.3, null}|{200, null}|
|0000502|            Bohemios| {1905, null}|         100|                  \n|{4.2, null}| {14, null}|
|0000502|            Bohemios| {1905, null}|         100|                  \n|{4.2, null}| {14, null}|
|0000574|The Story of the ...| {1906, null}|          70|action,adventure,...|{6.0, null}|{797, null}|
|0000574|The Story of the ...| {1906, null}|          70|action,adventure

In [10]:
# Filtro para incluir linhas que contenham "crime" na coluna "genero"
filtrado_crime = col('genero').like('%crime%')

# Aplicando filtro ao df
df_filtrado = df_movies.filter(filtrado_crime)

df_filtrado.show()

+-------+--------------------+-------------+------------+--------------+-----------+------------+
|     id|      titulopincipal|anolancamento|tempominutos|        genero|  notamedia| numerovotos|
+-------+--------------------+-------------+------------+--------------+-----------+------------+
|0001115|       Ansigttyven I| {1910, null}|          \N|         crime|{4.6, null}|  {19, null}|
|0001115|       Ansigttyven I| {1910, null}|          \N|         crime|{4.6, null}|  {19, null}|
|0001115|       Ansigttyven I| {1910, null}|          \N|         crime|{4.6, null}|  {19, null}|
|0001115|       Ansigttyven I| {1910, null}|          \N|         crime|{4.6, null}|  {19, null}|
|0001115|       Ansigttyven I| {1910, null}|          \N|         crime|{4.6, null}|  {19, null}|
|0001115|       Ansigttyven I| {1910, null}|          \N|         crime|{4.6, null}|  {19, null}|
|0001115|       Ansigttyven I| {1910, null}|          \N|         crime|{4.6, null}|  {19, null}|
|0001115|       Ansi

In [28]:
# Obtendo o número de linhas (registros)
num_linhas = df_filtrado.count()
# Obtendo o número de colunas
num_colunas = len(df_filtrado.columns)

# Exiba o número de linhas e colunas
print("Número de Linhas:", num_linhas)
print("Número de Colunas:", num_colunas)

Número de Linhas: 102132
Número de Colunas: 7


In [29]:
# lista com expressão para calcular a contagem de nulos em cada coluna
expressao_contagem_nulos = [count(when(col(coluna).isNull(), coluna)).alias(coluna) for coluna in df_filtrado.columns]

# função select para calcular a contagem de nulos em cada coluna
contagem_nulos = df_filtrado.select(expressao_contagem_nulos)

# Exiba a contagem de nulos
contagem_nulos.show()

+---+--------------+-------------+------------+------+---------+-----------+
| id|titulopincipal|anolancamento|tempominutos|genero|notamedia|numerovotos|
+---+--------------+-------------+------------+------+---------+-----------+
|  0|             0|            0|           0|     0|        0|          0|
+---+--------------+-------------+------------+------+---------+-----------+


#### Os campos com valores inconsistentes são transformados em valores nulos no código a seguir

In [12]:
# Lista de todas as colunas do df
colunas = df_filtrado.columns

# Iterando sobre todas as colunas e aplicando substituição por valor nulo
for coluna in colunas:
    df_filtrado = df_filtrado.withColumn(
        coluna,
        when(
            (col(coluna).cast('string') == '\\N') | (col(coluna).cast('string') == '\\n'),
            None
        ).otherwise(col(coluna))
    )

In [13]:
# lista com expressão para calcular a contagem de nulos em cada coluna
expressao_contagem_nulos_2 = [count(when(col(coluna).isNull(), coluna)).alias(coluna) for coluna in df_filtrado.columns]

# função select para calcular a contagem de nulos em cada coluna
contagem_nulos = df_filtrado.select(expressao_contagem_nulos_2)

# Exiba a contagem de nulos
contagem_nulos.show()

+---+--------------+-------------+------------+------+---------+-----------+
| id|titulopincipal|anolancamento|tempominutos|genero|notamedia|numerovotos|
+---+--------------+-------------+------------+------+---------+-----------+
|  0|             0|            0|        6393|     0|        0|          0|
+---+--------------+-------------+------------+------+---------+-----------+


#### Example: Após a limpeza dos dados, eles são salvos no formato parquet na pasta "Trusted" no "S3" e são criados seu database e table no "Glue Data Catalog"

In [16]:
# Criando um DynamicFrame a partir do df
dyf_filtrado = DynamicFrame.fromDF(df_filtrado, glueContext, "dyf_filtrado")

s3output = glueContext.getSink(
  path="s3://data-lake-daniele/Trusted/dados-do-csv/",
  connection_type="s3",
  updateBehavior="UPDATE_IN_DATABASE",
  compression="snappy",
  enableUpdateCatalog=True,
  transformation_ctx="s3output",
)
s3output.setCatalogInfo(
  catalogDatabase="movies-trusted-data", catalogTableName="dados-do-csv"
)
s3output.setFormat("glueparquet")
s3output.writeFrame(dyf_filtrado)